## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Sep. 22nd Sep. 29th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,September-27-2024,Condo/Co-op,1951 NE 39th St #128,Lighthouse Point,FL,33064.0,195000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lighthouse-Point/195...,Beaches MLS,RX-11024552,N,Y,26.280567,-80.095544


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
96,PAST SALE,2024-09-30,Condo/Co-op,3095 N Course Dr #401,Pompano Beach,FL,33069.0,265000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/3095-N...,Beaches MLS,F10430536,N,Y,26.227397,-80.163760
74,PAST SALE,2024-09-30,Condo/Co-op,3333 NE 34th St #1408,Fort Lauderdale,FL,33308.0,425000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3333...,Beaches MLS,F10449429,N,Y,26.170540,-80.101135
26,PAST SALE,2024-09-30,Condo/Co-op,4004 Swansea A,Deerfield Beach,FL,33442.0,172000.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/4004...,Beaches MLS,RX-11008219,N,Y,26.314515,-80.129391
121,PAST SALE,2024-09-30,Condo/Co-op,3101 Bayshore Dr #901,Fort Lauderdale,FL,33304.0,660000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3101...,Beaches MLS,F10412440,N,Y,26.128581,-80.103951
118,PAST SALE,2024-09-27,Condo/Co-op,2240 N Cypress Bend Dr #105,Pompano Beach,FL,33069.0,265000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2240-N...,Beaches MLS,F10419471,N,Y,26.217585,-80.154552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,PAST SALE,NaT,Condo/Co-op,2802 N 46th Ave Unit 514B,Hollywood,FL,33021.0,540000.0,2.0,3.0,...,NaN,NaN,NaN,https://www.redfin.com/FL/Hollywood/2802-N-46t...,NaN,NaN,N,Y,26.034950,-80.191114
123,PAST SALE,NaT,Condo/Co-op,1719 Wiley St #11,Hollywood,FL,33020.0,100000.0,1.0,1.0,...,NaN,NaN,NaN,https://www.redfin.com/FL/Hollywood/1719-Wiley...,NaN,NaN,N,Y,25.999862,-80.141243
124,PAST SALE,NaT,Condo/Co-op,953 NW 93rd Ave,Plantation,FL,33324.0,325000.0,3.0,2.0,...,NaN,NaN,NaN,https://www.redfin.com/FL/Plantation/953-NW-93...,NaN,NaN,N,Y,26.134535,-80.272220
125,PAST SALE,NaT,Condo/Co-op,4881 NW 22nd St Unit 18B,Lauderhill,FL,33313.0,105000.0,2.0,2.0,...,NaN,NaN,NaN,https://www.redfin.com/FL/Lauderhill/4881-NW-2...,NaN,NaN,N,Y,26.154306,-80.216079


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

126


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$343,326


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$275


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$43,259,100


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [22]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input county name:
Broward County
Input Previous Week Condo Sales Total (number units sold):
73
Input Previous Week Condo Average Sales Price:
276614
Input Previous Week Condo Average PSF:
261
Input Previous Week Condo Sales total (ex: 198_000_000)
20_190_000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [23]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [24]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [25]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [26]:
pd.set_option('display.max_columns',None)

In [27]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-09-25,Condo/Co-op,525 N Ft Lauderdale Bch Blvd #1604,Fort Lauderdale,FL,33304.0,3650000.0,2.0,3.0,Four Seasons,2069.0,NaN,2022.0,NaN,1764.0,6788.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Beaches MLS,F10419239,N,Y,26.130123,-80.103443,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Daniel Teixeira,Douglas Elliman,Enrique Teixeira,Douglas Elliman,None,"Excel Realty, Inc.",None,None,1,orange


In [28]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [29]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-09-25,Condo/Co-op,525 N Ft Lauderdale Bch Blvd #1604,Fort Lauderdale,FL,33304.0,3650000.0,2.0,3.0,Four Seasons,2069.0,NaN,2022.0,NaN,1764.0,6788.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Beaches MLS,F10419239,N,Y,26.130123,-80.103443,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Daniel Teixeira,Douglas Elliman,Enrique Teixeira,Douglas Elliman,None,"Excel Realty, Inc.",None,None,1,orange
1,PAST SALE,2024-09-27,Condo/Co-op,100 S Birch Rd #2504,Fort Lauderdale,FL,33316.0,1330000.0,3.0,2.5,Jackson Tower Las Olas,1934.0,NaN,2001.0,NaN,688.0,2432.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10453488,N,Y,26.121167,-80.106486,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Jolanta Johnson,"Compass Florida, LLC",Marzena Annis,"Compass Florida, LLC",Gina Concepcion,Guardian Real Estate Co.,None,None,2,blue
2,PAST SALE,2024-09-23,Condo/Co-op,6051 N Ocean Dr #1104,Hollywood,FL,33019.0,1230000.0,3.0,2.5,Renaissance On The Ocean,2233.0,NaN,1999.0,NaN,551.0,2721.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,Beaches MLS,F10451839,N,Y,26.052035,-80.113046,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,Beth Maitlen,Hollywood Beach Realty Inc,Tara Leone,Hollywood Beach Realty Inc,Victor Montello,Tomlinson Realty Group,None,None,3,blue
3,PAST SALE,NaT,Condo/Co-op,1 Las Olas Cir #501,Fort Lauderdale,FL,33316.0,1200000.0,3.0,3.0,NaN,1925.0,NaN,1970.0,NaN,623.0,NaN,NaN,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,NaN,NaN,N,Y,26.118076,-80.107100,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Shirley Smith,Intercoastal Realty Inc,None,None,Shirley Smith,Intercoastal Realty Inc,None,None,4,blue
4,PAST SALE,2024-09-23,Condo/Co-op,2100 S Ocean Ln #512,Fort Lauderdale,FL,33316.0,980000.0,2.0,2.0,Point Of Americas,1813.0,NaN,1969.0,NaN,541.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Beaches MLS,F10457658,N,Y,26.096516,-80.106815,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,Lina Buitrago,"Compass Florida, LLC",None,None,5,blue
5,PAST SALE,2024-09-24,Condo/Co-op,3750 Galt Ocean Dr #211,Fort Lauderdale,FL,33308.0,950000.0,3.0,2.5,Regency Tower South Condo,2323.0,NaN,1970.0,NaN,409.0,2011.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Beaches MLS,F10445089,N,Y,26.174061,-80.097794,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Howard Goldberg PA,RE/MAX Consultants Realty 1,None,None,Nancy Seltzer,S & B Realty,Justin Bartholomew,S & B Realty,6,blue
6,PAST SALE,2024-09-25,Condo/Co-op,2880 NE 14th Street Cswy #1012,Pompano Beach,FL,33062.0,817000.0,2.0,2.0,The Pointe,1800.0,NaN,1999.0,NaN,454.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Beaches MLS,F10441538,N,Y,26.248695,-80.091423,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Stuart Goodwin,Century 21 Hansen Realty Inc,None,None,Meghan Stewart,Compass Florida LLC,None,None,7,blue
7,PAST SALE,2024-09-24,Condo/Co-op,2847 Kinsington Cir Unit 16-4,Weston,FL,33332.0,815000.0,3.0,2.5,LEGENDS AT WESTON HILLS,2290.0,NaN,2002.0,NaN,356.0,800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Weston/2847-Kinsingt...,MARMLS,A11629112,N,Y,26.093084,-80.385574,https://www.redfin.com/FL/Weston/2847-Kinsingt...,Karen Kroboth,One Sotheby's Int'l Realty,None,None,Carlos Uzcategui,United Realty Group Inc,None,None,8,blue
8,PAST SALE,2024-09-26,Condo/Co-op,2509 N Ocean Blvd #777,Fort Lauderdale,FL,33305.0,750000.0,2.0,2.0,Fountains O

In [30]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [31]:
m.save('index.html')

## Data snagger

In [32]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [33]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-09-25,Condo/Co-op,525 N Ft Lauderdale Bch Blvd #1604,Fort Lauderdale,FL,33304.0,3650000.0,2.0,3.0,Four Seasons,2069.0,NaN,2022.0,NaN,1764.0,6788.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Beaches MLS,F10419239,N,Y,26.130123,-80.103443,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Daniel Teixeira,Douglas Elliman,Enrique Teixeira,Douglas Elliman,None,"Excel Realty, Inc.",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [34]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/BC_County_condo_sales_week_ending_09302024


# CREATE TEMPLATE 

In [35]:
muni_set = set(df_top_ten['CITY'])

In [36]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [37]:
df_top_ten.reset_index(inplace=True,drop=True)

In [38]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [39]:
top_sale

'Four Seasons at 525 N Ft Lauderdale Bch Blvd #1604 in Fort Lauderdale'

In [40]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [41]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [42]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-09-25,Condo/Co-op,525 N Ft Lauderdale Bch Blvd #1604,Fort Lauderdale,FL,33304.0,3650000.0,2.0,3.0,Four Seasons,2069.0,NaN,2022.0,NaN,1764.0,6788.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Beaches MLS,F10419239,N,Y,26.130123,-80.103443,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Daniel Teixeira,Douglas Elliman,Enrique Teixeira,Douglas Elliman,None,"Excel Realty, Inc.",None,None,1,orange,Four Seasons at 525 N Ft Lauderdale Bch Blvd #...
1,PAST SALE,2024-09-27,Condo/Co-op,100 S Birch Rd #2504,Fort Lauderdale,FL,33316.0,1330000.0,3.0,2.5,Jackson Tower Las Olas,1934.0,NaN,2001.0,NaN,688.0,2432.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10453488,N,Y,26.121167,-80.106486,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Jolanta Johnson,"Compass Florida, LLC",Marzena Annis,"Compass Florida, LLC",Gina Concepcion,Guardian Real Estate Co.,None,None,2,blue,Jackson Tower Las Olas at 100 S Birch Rd #2504...
2,PAST SALE,2024-09-23,Condo/Co-op,6051 N Ocean Dr #1104,Hollywood,FL,33019.0,1230000.0,3.0,2.5,Renaissance On The Ocean,2233.0,NaN,1999.0,NaN,551.0,2721.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,Beaches MLS,F10451839,N,Y,26.052035,-80.113046,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,Beth Maitlen,Hollywood Beach Realty Inc,Tara Leone,Hollywood Beach Realty Inc,Victor Montello,Tomlinson Realty Group,None,None,3,blue,Renaissance On The Ocean at 6051 N Ocean Dr #1...
3,PAST SALE,NaT,Condo/Co-op,1 Las Olas Cir #501,Fort Lauderdale,FL,33316.0,1200000.0,3.0,3.0,NaN,1925.0,NaN,1970.0,NaN,623.0,NaN,NaN,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,NaN,NaN,N,Y,26.118076,-80.107100,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Shirley Smith,Intercoastal Realty Inc,None,None,Shirley Smith,Intercoastal Realty Inc,None,None,4,blue,NaN
4,PAST SALE,2024-09-23,Condo/Co-op,2100 S Ocean Ln #512,Fort Lauderdale,FL,33316.0,980000.0,2.0,2.0,Point Of Americas,1813.0,NaN,1969.0,NaN,541.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Beaches MLS,F10457658,N,Y,26.096516,-80.106815,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,Lina Buitrago,"Compass Florida, LLC",None,None,5,blue,Point Of Americas at 2100 S Ocean Ln #512 in F...
5,PAST SALE,2024-09-24,Condo/Co-op,3750 Galt Ocean Dr #211,Fort Lauderdale,FL,33308.0,950000.0,3.0,2.5,Regency Tower South Condo,2323.0,NaN,1970.0,NaN,409.0,2011.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Beaches MLS,F10445089,N,Y,26.174061,-80.097794,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Howard Goldberg PA,RE/MAX Consultants Realty 1,None,None,Nancy Seltzer,S & B Realty,Justin Bartholomew,S & B Realty,6,blue,Regency Tower South Condo at 3750 Galt Ocean D...
6,PAST SALE,2024-09-25,Condo/Co-op,2880 NE 14th Street Cswy #1012,Pompano Beach,FL,33062.0,817000.0,2.0,2.0,The Pointe,1800.0,NaN,1999.0,NaN,454.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Beaches MLS,F10441538,N,Y,26.248695,-80.091423,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Stuart Goodwin,Century 21 Hansen Realty Inc,None,None,Meghan Stewart,Compass Florida LLC,None,None,7,blue,The Pointe at 2880 NE 14th Street Cswy #1012 i...
7,PAST SALE,2024-09-24,Condo/Co-op,2847 Kinsington Cir Unit 16-4,Weston,FL,33332.0,815000.0,3.0,2.5,LEGENDS AT WESTON HILLS,2290.0,NaN,2002.0,NaN,356.0,800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Weston/2847-Kinsingt...,

In [43]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Four Seasons closing tops Broward County weekly condo sales 
DEK: Top 10 sales ranged from $700,000M to $3,650,000M
FEATURED HED:
SEO HED: Broward County County Weekly Condo Report 
SEO DESCRIPTION: Broward County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward County, weekly condo sales, Hollywood, Weston, Pompano Beach, Fort Lauderdale

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County County’s NEWS PEG HERE.

Brokers closed 126 condo sales totaling $43,259,100 million from Sep. 22nd to Sep. 29th. The previous week, brokers closed 73 condo sales totaling $20,190,000.

Last week’s units sold for an average of $343,326, higher than the $276,614 average price from the previous week. The averag

In [44]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [45]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [46]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [47]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-09-25,Condo/Co-op,525 N Ft Lauderdale Bch Blvd #1604,Fort Lauderdale,FL,33304.0,3650000.0,2.0,3.0,Four Seasons,2069.0,NaN,2022.0,NaN,1764.0,6788.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Beaches MLS,F10419239,N,Y,26.130123,-80.103443,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Daniel Teixeira,Douglas Elliman,Enrique Teixeira,Douglas Elliman,None,"Excel Realty, Inc.",None,None,1,orange,Four Seasons at 525 N Ft Lauderdale Bch Blvd #...
1,PAST SALE,2024-09-27,Condo/Co-op,100 S Birch Rd #2504,Fort Lauderdale,FL,33316.0,1330000.0,3.0,2.5,Jackson Tower Las Olas,1934.0,NaN,2001.0,NaN,688.0,2432.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10453488,N,Y,26.121167,-80.106486,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Jolanta Johnson,"Compass Florida, LLC",Marzena Annis,"Compass Florida, LLC",Gina Concepcion,Guardian Real Estate Co.,None,None,2,blue,Jackson Tower Las Olas at 100 S Birch Rd #2504...
2,PAST SALE,2024-09-23,Condo/Co-op,6051 N Ocean Dr #1104,Hollywood,FL,33019.0,1230000.0,3.0,2.5,Renaissance On The Ocean,2233.0,NaN,1999.0,NaN,551.0,2721.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,Beaches MLS,F10451839,N,Y,26.052035,-80.113046,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,Beth Maitlen,Hollywood Beach Realty Inc,Tara Leone,Hollywood Beach Realty Inc,Victor Montello,Tomlinson Realty Group,None,None,3,blue,Renaissance On The Ocean at 6051 N Ocean Dr #1...
3,PAST SALE,NaT,Condo/Co-op,1 Las Olas Cir #501,Fort Lauderdale,FL,33316.0,1200000.0,3.0,3.0,NaN,1925.0,NaN,1970.0,NaN,623.0,NaN,NaN,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,NaN,NaN,N,Y,26.118076,-80.107100,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Shirley Smith,Intercoastal Realty Inc,None,None,Shirley Smith,Intercoastal Realty Inc,None,None,4,blue,NaN
4,PAST SALE,2024-09-23,Condo/Co-op,2100 S Ocean Ln #512,Fort Lauderdale,FL,33316.0,980000.0,2.0,2.0,Point Of Americas,1813.0,NaN,1969.0,NaN,541.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Beaches MLS,F10457658,N,Y,26.096516,-80.106815,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,Lina Buitrago,"Compass Florida, LLC",None,None,5,blue,Point Of Americas at 2100 S Ocean Ln #512 in F...
5,PAST SALE,2024-09-24,Condo/Co-op,3750 Galt Ocean Dr #211,Fort Lauderdale,FL,33308.0,950000.0,3.0,2.5,Regency Tower South Condo,2323.0,NaN,1970.0,NaN,409.0,2011.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Beaches MLS,F10445089,N,Y,26.174061,-80.097794,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Howard Goldberg PA,RE/MAX Consultants Realty 1,None,None,Nancy Seltzer,S & B Realty,Justin Bartholomew,S & B Realty,6,blue,Regency Tower South Condo at 3750 Galt Ocean D...
6,PAST SALE,2024-09-25,Condo/Co-op,2880 NE 14th Street Cswy #1012,Pompano Beach,FL,33062.0,817000.0,2.0,2.0,The Pointe,1800.0,NaN,1999.0,NaN,454.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Beaches MLS,F10441538,N,Y,26.248695,-80.091423,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Stuart Goodwin,Century 21 Hansen Realty Inc,None,None,Meghan Stewart,Compass Florida LLC,None,None,7,blue,The Pointe at 2880 NE 14th Street Cswy #1012 i...
7,PAST SALE,2024-09-24,Condo/Co-op,2847 Kinsington Cir Unit 16-4,Weston,FL,33332.0,815000.0,3.0,2.5,LEGENDS AT WESTON HILLS,2290.0,NaN,2002.0,NaN,356.0,800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Weston/2847-Kinsingt...,

In [48]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: Four Seasons closing tops Broward County weekly condo sales 
DEK: Top 10 sales ranged from $700,000M to $3,650,000M
FEATURED HED:
SEO HED: Broward County County Weekly Condo Report 
SEO DESCRIPTION: Broward County County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward County condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward County, weekly condo sales, Hollywood, Weston, Pompano Beach, Fort Lauderdale

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County County’s NEWS PEG HERE.

Brokers closed 126 condo sales totaling $43,259,100 million from Sep. 22nd to Sep. 29th. The previous week, brokers closed 73 condo sales totaling $20,190,000.

Last week’s units sold for an average of $343,326, higher than the $276,614 average price from the previous week. The averag

In [56]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Fort-Lauderdale/525-N-Fort-Lauderdale-Beach-Blvd-33304/unit-1604/home/174573453


In [63]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Fort-Lauderdale/100-S-Birch-Rd-33316/unit-2504/home/104097591


In [64]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Hollywood/777-N-Ocean-Dr-33019/unit-N526/home/145953462


In [65]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Fort-Lauderdale/2509-N-Ocean-Blvd-33305/unit-777/home/42082098


In [66]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Fort-Lauderdale/525-N-Fort-Lauderdale-Beach-Blvd-33304/unit-1604/home/174573453


In [67]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Weston/2847-Kinsington-Cir-33332/unit-16-4/home/42005978


In [68]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-09-25,Condo/Co-op,525 N Ft Lauderdale Bch Blvd #1604,Fort Lauderdale,FL,33304.0,3650000.0,2.0,3.0,Four Seasons,2069.0,NaN,2022.0,NaN,1764.0,6788.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Beaches MLS,F10419239,N,Y,26.130123,-80.103443,https://www.redfin.com/FL/Fort-Lauderdale/525-...,Daniel Teixeira,Douglas Elliman,Enrique Teixeira,Douglas Elliman,None,"Excel Realty, Inc.",None,None,1,orange,Four Seasons at 525 N Ft Lauderdale Bch Blvd #...
1,PAST SALE,2024-09-27,Condo/Co-op,100 S Birch Rd #2504,Fort Lauderdale,FL,33316.0,1330000.0,3.0,2.5,Jackson Tower Las Olas,1934.0,NaN,2001.0,NaN,688.0,2432.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10453488,N,Y,26.121167,-80.106486,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Jolanta Johnson,"Compass Florida, LLC",Marzena Annis,"Compass Florida, LLC",Gina Concepcion,Guardian Real Estate Co.,None,None,2,blue,Jackson Tower Las Olas at 100 S Birch Rd #2504...
2,PAST SALE,2024-09-23,Condo/Co-op,6051 N Ocean Dr #1104,Hollywood,FL,33019.0,1230000.0,3.0,2.5,Renaissance On The Ocean,2233.0,NaN,1999.0,NaN,551.0,2721.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,Beaches MLS,F10451839,N,Y,26.052035,-80.113046,https://www.redfin.com/FL/Hollywood/6051-N-Oce...,Beth Maitlen,Hollywood Beach Realty Inc,Tara Leone,Hollywood Beach Realty Inc,Victor Montello,Tomlinson Realty Group,None,None,3,blue,Renaissance On The Ocean at 6051 N Ocean Dr #1...
3,PAST SALE,NaT,Condo/Co-op,1 Las Olas Cir #501,Fort Lauderdale,FL,33316.0,1200000.0,3.0,3.0,NaN,1925.0,NaN,1970.0,NaN,623.0,NaN,NaN,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,NaN,NaN,N,Y,26.118076,-80.107100,https://www.redfin.com/FL/Fort-Lauderdale/1-La...,Shirley Smith,Intercoastal Realty Inc,None,None,Shirley Smith,Intercoastal Realty Inc,None,None,4,blue,NaN
4,PAST SALE,2024-09-23,Condo/Co-op,2100 S Ocean Ln #512,Fort Lauderdale,FL,33316.0,980000.0,2.0,2.0,Point Of Americas,1813.0,NaN,1969.0,NaN,541.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Beaches MLS,F10457658,N,Y,26.096516,-80.106815,https://www.redfin.com/FL/Fort-Lauderdale/2100...,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,Lina Buitrago,"Compass Florida, LLC",None,None,5,blue,Point Of Americas at 2100 S Ocean Ln #512 in F...
5,PAST SALE,2024-09-24,Condo/Co-op,3750 Galt Ocean Dr #211,Fort Lauderdale,FL,33308.0,950000.0,3.0,2.5,Regency Tower South Condo,2323.0,NaN,1970.0,NaN,409.0,2011.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Beaches MLS,F10445089,N,Y,26.174061,-80.097794,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Howard Goldberg PA,RE/MAX Consultants Realty 1,None,None,Nancy Seltzer,S & B Realty,Justin Bartholomew,S & B Realty,6,blue,Regency Tower South Condo at 3750 Galt Ocean D...
6,PAST SALE,2024-09-25,Condo/Co-op,2880 NE 14th Street Cswy #1012,Pompano Beach,FL,33062.0,817000.0,2.0,2.0,The Pointe,1800.0,NaN,1999.0,NaN,454.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Beaches MLS,F10441538,N,Y,26.248695,-80.091423,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Stuart Goodwin,Century 21 Hansen Realty Inc,None,None,Meghan Stewart,Compass Florida LLC,None,None,7,blue,The Pointe at 2880 NE 14th Street Cswy #1012 i...
7,PAST SALE,2024-09-24,Condo/Co-op,2847 Kinsington Cir Unit 16-4,Weston,FL,33332.0,815000.0,3.0,2.5,LEGENDS AT WESTON HILLS,2290.0,NaN,2002.0,NaN,356.0,800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Weston/2847-Kinsingt...,

## Time on Market Calculator

In [70]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 7, 30) ## List (Earlier) date
date2 = datetime(2024, 9, 27) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

59


## Clean CSV for Datawrapper Chart

In [71]:
chart_df = df_top_ten

In [72]:
chart_df = chart_df.fillna(" ")

In [73]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [74]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [75]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [76]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [77]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

ValueError: Unknown format code 'f' for object of type 'str'

In [78]:
csv_date_string = today.strftime("%m_%d_%Y")

In [79]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")